In [1]:
import inspect
import time
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
import os

from ray.tune.registry import register_env
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper
import warnings
import numpy as np
from ray import air, tune
warnings.filterwarnings('ignore')

In [4]:
def env_creator(env_config: dict):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
    agents = {"Red": B_lineAgent, "Green": GreenAgent}
    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
    return env

def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")

register_env(name="CybORG", env_creator=env_creator)

In [13]:

tune.Tuner(
        "PPO",
        run_config=air.RunConfig(
            stop={"timesteps_total": 2e6},
            local_dir='results/APPO', name="explore2",
            checkpoint_config=air.CheckpointConfig(
                checkpoint_frequency=500, 
            ),
        ),
        param_space={
            # CC3 specific.
            "env": "CybORG",
            # General
            "num_gpus": 1,
            "num_workers": 30,
            "horizon": 100,
            "num_envs_per_worker": 1,
            #"exploration_config": tune.grid_search([{"type": "RE3",
            #    "embeds_dim": 128,
            #    "beta_schedule": "constant",
            #    "sub_exploration": {
            #        "type": "StochasticSampling",
            #},}, 
            "exploration_config": {"type": "StochasticSampling"},
            #algo params
            "train_batch_size": 3000,
            "lr": 0.0005,
            "gamma": 0.95,
            "framework": 'tf',
            "model": {
                    "fcnet_hiddens": [512, 512],
                    "fcnet_activation": "relu",
                    #'use_lstm': True,
                    #'max_seq_len': 5,
                },
            "output": "dataset",
            "output_config": {
                "format": "json",
                "path": "logs"},
            "output_compress_columns": ['prev_actions', 'prev_rewards', 'dones', 't', 'eps_id', 'unroll_id', 'agent_index', 'action_prob', 'action_logp', 'action_dist_inputs', 'advantages', 'value_targets']
        },
    ).fit()

(PPO pid=35306) 2022-12-16 12:34:07,417	INFO algorithm.py:2303 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPO pid=35306) 2022-12-16 12:34:07,418	WARNING ppo.py:351 -- `train_batch_size` (3000) cannot be achieved with your other settings (num_workers=30 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 100.
(PPO pid=35306) 2022-12-16 12:34:07,418	INFO ppo.py:379 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPO pid=35306) 2022-12-16 12:34:07,419	INFO algorithm.py:457 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=35336) 2022-12-16 12:34

Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_recreated_workers,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_CybORG_eb02e_00000,261000,"{'num_env_steps_sampled': 261000, 'num_env_steps_trained': 261000, 'num_agent_steps_sampled': 261000, 'num_agent_steps_trained': 261000}",{},2022-12-16_12-42-23,False,100,{},-15.4,-102.8,-227.3,30,2610,128860a1aad34cc29b334440d11a9248,01589170c3ff,"{'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.20000000298023224, 'cur_lr': 0.0005000000237487257, 'total_loss': 6.3965383, 'policy_loss': -0.1347836, 'vf_loss': 6.523888, 'vf_explained_var': 0.03908292, 'kl': 0.037166, 'entropy': 2.320218, 'entropy_coeff': 0.0, 'model': {}}, 'train': None}}, 'num_env_steps_sampled': 261000, 'num_env_steps_trained': 261000, 'num_agent_steps_sampled': 261000, 'num_agent_steps_trained': 261000}",87,172.28.0.2,261000,261000,261000,3000,261000,3000,0,30,0,3000,"{'cpu_util_percent': 33.300000000000004, 'ram_util_percent': 30.0}",35306,{},{},{},"{'mean_raw_obs_processing_ms': 1.9863312800918083, 'mean_inference_ms': 1.6620056666155616, 'mean_action_processing_ms': 0.1481255824981774, 'mean_env_wait_ms': 10.805737465934882, 'mean_env_render_ms': 0.0}","{'episode_reward_max': -15.399999999999968, 'episode_reward_min': -227.29999999999995, 'episode_reward_mean': -102.79999999999993, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 30, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-104.5, -120.19999999999996, -99.79999999999986, -17.999999999999993, -105.59999999999991, -54.70000000000007, -122.7999999999998, -96.90000000000003, -106.49999999999977, -99.69999999999985, -75.69999999999999, -23.300000000000004, -121.7999999999998, -186.59999999999968, -117.79999999999984, -87.1, -106.20000000000005, -41.700000000000024, -97.10000000000002, -85.6, -107.69999999999996, -71.70000000000002, -107.4, -56.1000000000001, -140.69999999999982, -99.00000000000004, -47.80000000000008, -165.2, -137.79999999999973, -96.40000000000003, -79.80000000000004, -108.69999999999996, -92.29999999999995, -105.5, -36.40000000000004, -66.90000000000002, -99.79999999999988, -32.800000000000075, -102.79999999999998, -140.19999999999985, -98.49999999999996, -102.79999999999998, -116.69999999999975, -47.80000000000003, -97.7999999999999, -97.79999999999995, -118.5, -123.79999999999981, -191.7999999999997, -129.79999999999978, -26.80000000000004, -222.79999999999964, -100.99999999999987, -33.50000000000009, -109.69999999999985, -116.20000000000002, -178.7999999999997, -120.89999999999984, -100.79999999999984, -26.600000000000076, -127.6999999999998, -227.29999999999995, -63.80000000000006, -120.39999999999984, -15.399999999999968, -143.29999999999995, -94.7999999999999, -35.30000000000008, -129.79999999999976, -39.6, -116.39999999999976, -148.79999999999978, -90.79999999999987, -139.7999999999997, -111.79999999999984, -106.90000000000003, -86.0, -53.80000000000002, -111.79999999999983, -85.79999999999993, -120.79999999999977, -122.09999999999994, -74.9, -182.79999999999959, -98.60000000000001, -58.8000000000001, -119.7999999999998, -120.79999999999987, -109.79999999999986, -112.10000000000002, -30.40000000000002, -144.79999999999978, -160.29999999999998, -108.9, -105.79999999999988, -111.6, -164.79999999999984, -1

2022-12-16 12:42:24,713	WARNING tune.py:705 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2022-12-16 12:42:28,988	ERROR tune.py:773 -- Trials did not complete: [PPO_CybORG_eb02e_00000]
2022-12-16 12:42:28,989	INFO tune.py:777 -- Total run time: 508.23 seconds (507.95 seconds for the tuning loop).
2022-12-16 12:42:28,990	WARNING tune.py:783 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [12]:
!ls /

NGC-DL-CONTAINER-LICENSE  boot	home   lib64   mnt   root  srv	tmp
app			  dev	lib    libx32  opt   run   sys	usr
bin			  etc	lib32  media   proc  sbin  tf	var


In [ ]:
!pip show ray

In [ ]:
path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
agents = {"Red": B_lineAgent, "Green": GreenAgent}
cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)

In [ ]:
env.observation_space

In [18]:
dist = np.array([0.9997, 0.0001, 0.0001, 0.0001])
entropy = -np.sum(dist * np.log(dist))
entropy

0.0030630571070921465

In [19]:
dist = np.array([0.5, 0.5, 0.0001, 0.0001])
entropy = -np.sum(dist * np.log(dist))
entropy

0.6949892486343404

In [20]:
dist = np.array([0.25, 0.25, 0.25, 0.25])
entropy = -np.sum(dist * np.log(dist))
entropy

1.3862943611198906